In [1]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer
import re

class EmotionClassifier:
    def __init__(self, model_path="monologg/kobert", num_labels=7, device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model.to(self.device)

        # Mapping numeric labels to emotion names
        self.label_to_emotion = {
            0: "중립",
            1: "놀람",
            2: "분노",
            3: "슬픔",
            4: "행복",
            5: "혐오",
            6: "공포"
        }

    def load_model(self, model_file):
        """학습된 감정분류 모델 불러오기"""
        self.model.load_state_dict(torch.load(model_file))

    def preprocess_text(self, text):
        """텍스트 전처리"""
        return re.sub("[^0-9a-zA-Z가-힣\s+]", "", text)

    def predict_emotion(self, text):
        """감정 분류 및 예측"""
        # Preprocess and tokenize input text
        cleaned_text = self.preprocess_text(text)
        encoded_input = self.tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}
        
        # Get prediction
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**encoded_input)
            predicted_label = outputs.logits.argmax(dim=1).item()
        
        # Map label to emotion name
        predicted_emotion = self.label_to_emotion[predicted_label]
        
        return predicted_emotion


In [2]:
# Example Usage
if __name__ == "__main__": # 직접 실행될때만 실행되게. 클래스를 불러와서 사용될때 같이 실행안되게
    classifier = EmotionClassifier()
    classifier.load_model("new_data_test.pth")
    sample_text = "너 뭐하는 애야 ?"
    emotion_label = classifier.predict_emotion(sample_text)
    print(f"Predicted Emotion Label: {emotion_label}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1946866/2998071824.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We 

Predicted Emotion Label: 놀람
